In [1]:
from langchain_chroma.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# Initialize embeddings model (ensure the same model used during creation)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Load the existing Chroma vector store
persist_directory = "../db/swim_rules"
vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

# Function to perform semantic search
def semantic_search(query, k=4):
    """
    Perform a semantic search on the Chroma vector store.

    Args:
        query (str): The query string to search for.
        k (int): The number of top results to retrieve.

    Returns:
        list: A list of documents matching the query.
    """
    results = vector_store.similarity_search_with_score(query, k=k,)
    
    for chunk, result in enumerate(results):
        doc, score = result
        print(f">>>Chunk: {chunk+1} score: {score:0.4f} Page {doc.metadata['page']}: {doc.page_content}...")
    return results

# Example usage
queries = ["What are the swimming rules for freestyle?",
            "what is the turn rules for breaststroke?",
            "can a swimmer return to the wall after making a turn?"]

for query in queries:
    print(f"\nQuery: {query}")
    print(semantic_search(query))


Query: What are the swimming rules for freestyle?
>>>Chunk: 1 score: 0.6649 Page 24: 23
1
101.4
swimmer has passed the final 5-meter mark (long course and short course 
meters) or 5-yard mark (short course yards), immediately prior to reaching for the 
finish, the swimmer may be completely submerged prior to the touch.
.3  Turns — Upon completion of each length, some part of the swimmer must touch 
the wall. During the turn the shoulders may be turned past the vertical toward the 
breast after which an immediate continuous single arm pull or immediate continu-
ous simultaneous double arm pull may be used to initiate the turn. The swimmer 
must have returned to a position on the back upon leaving the wall.
.4  Finish — Upon the finish of the race, the swimmer must touch the wall while on 
the back.
101.5  FREESTYLE...
>>>Chunk: 2 score: 0.6835 Page 24: must have returned to a position on the back upon leaving the wall.
.4  Finish — Upon the finish of the race, the swimmer must touch th